In [21]:
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import random
import json
import os
import shutil
from pathlib import Path
import os.path
import subprocess
import os
import glob
import sys
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

def coco_to_yolo_bb(x1, y1, w, h, image_w, image_h):
    return [((2*x1 + w)/(2*image_w)) , ((2*y1 + h)/(2*image_h)), w/image_w, h/image_h]

def mkdir(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)

def get_cat_data():
    cats = coco.loadCats(coco.getCatIds())
    cat_map = {}
    for i,cat in enumerate(cats):
        cat_map[cat['id']] = i
    catIds = coco.getCatIds()
    return catIds, cat_map

def get_random_img_ids(numImgs):
    imgIds = coco.getImgIds()
    random.shuffle(imgIds)
    imgIds = list(filter(img_id_has_annotation, imgIds))
    imgIds = list(filter(img_id_has_positive_bboxes, imgIds))
    #add a filter for labels outside frame
    #have a discussion about these images probably
    return imgIds[:numImgs]

def img_id_has_positive_bboxes(imgId):
    annIds = coco.getAnnIds(imgIds=imgId)
    anns = coco.loadAnns(annIds)
    for ann in anns:
        negative_bbox = False 
        for bbox_val in ann['bbox']:
            if bbox_val < 0:
                negative_bbox = True
        if negative_bbox == True:
            return False
    return True

def img_id_has_annotation(imgId):
    annIds = coco.getAnnIds(imgIds=imgId)
    if(len(annIds) > 0):
        return True
    else:
        return False

def write_label_files(numImgs, cat_map, catIds, label_dir1):
    """
    Some images:
    *Do not have annotations.
    *Have negative bboxes
    """
    imgIds = get_random_img_ids(numImgs)

    for imgId in imgIds:
        img = coco.loadImgs(imgId)[0] #If only 1 imgId, len() = 1
        label_dir = Path(label_dir1)
        filename = img['file_name'].split('.')[0] + '.txt'
        with open(label_dir/filename, 'w') as file:
            annIds = coco.getAnnIds(imgIds = imgId, catIds=catIds)
            anns = coco.loadAnns(annIds)
            for i,ann in enumerate(anns):
                x_center, y_center, width, height = coco_to_yolo_bb(*ann['bbox'], img['width'], img['height'])
                file.write(f'{cat_map[ann["category_id"]]} {x_center} {y_center} {width} {height}\n')
    return

def copy_images_to_dataset_dir(src_image_dir, dest_image_dir, image_extension, src_label_dir):
    """
    x number of labels are written to the labels dir by write_label_files(x)
    This function copies the corresponding images into the sibling images dir, completing the dataset folder for YOLO training
    """
    fs = os.listdir(src_label_dir)
    for f in fs:
        if f == '.DS_Store':
            continue
        src = str(src_image_dir/f.split('.')[0]) + image_extension
        dest = str(dest_image_dir/ f.split('.')[0]) + image_extension
        if os.path.isfile(src):
            shutil.copyfile(src, dest)
        else:
            print(f'{src} is not a file')
    return

def remove_files(img_dir, label_dir):
    files = glob.glob(f'{img_dir}*')
    for f in files:
        os.remove(f)
    files = glob.glob(f'{label_dir}*')
    for f in files:
        os.remove(f)

def produce_nightowls_training_set(num_imgs):
    src_img_dir = '/usr/src/raw_datasets/nightowls/nightowls_training'
    label_dir =  f'/usr/src/datasets/source/n{num_imgs}t/labels/'
    img_dir = f'/usr/src/datasets/source/n{num_imgs}t/images/'

    os.system(f'mkdir -p {label_dir}')
    os.system(f'mkdir -p {img_dir}')

    print('Files removed')
    remove_files(img_dir,label_dir)

    catIds, cat_map = get_cat_data()
    write_label_files(num_imgs, cat_map, catIds, label_dir)
    copy_images_to_dataset_dir(Path(src_img_dir),Path(img_dir), '.png', label_dir) #dataset/images
    os.system(f'ls {img_dir} | wc -l ; ls {label_dir} | wc -l')

def produce_nightowls_validation_set(num_imgs):

    src_img_dir = '/usr/src/raw_datasets/nightowls/nightowls_validation'
    dataset_dir = f'/usr/src/datasets/source/n{num_imgs}v'
    label_dir =  f'{dataset_dir}/labels/'
    img_dir = f'{dataset_dir}/images/'

    shutil.rmtree(dataset_dir)
    print(f'{dataset_dir} removed')
    mkdir(dataset_dir)
    mkdir(label_dir)
    mkdir(img_dir)

    catIds, cat_map = get_cat_data()
    write_label_files(num_imgs, cat_map, catIds, label_dir)
    copy_images_to_dataset_dir(Path(src_img_dir),Path(img_dir), '.png', label_dir) #dataset/images

    os.system(f'ls {img_dir} | wc -l ; ls {label_dir} | wc -l')

def produce_coco_training_set(numImgs):
    label_dir =  '/Users/azakaria/Code/openmpf-yolo-training/coco/coco_train1/labels/'
    src_img_dir = '/Users/azakaria/Documents/yolo datasets/coco/train2017'
    img_dir = '/Users/azakaria/Code/openmpf-yolo-training/coco/coco_train1/images/'
    annotations_json_file = '/Users/azakaria/Documents/yolo datasets/coco/instances_train2017.json'

    remove_files(img_dir,label_dir)
    print('Files removed')

    #coco=COCO(annotations_json_file)
    catIds, cat_map = get_cat_data()
    write_label_files(numImgs, cat_map, catIds, label_dir) 
    copy_images_to_dataset_dir(Path(src_img_dir),Path(img_dir), '.jpg', label_dir) #dataset/images

    os.system(f'ls {img_dir} | wc -l ; ls {label_dir} | wc -l')
    #produce_coco_training_set(480)

def produce_coco_validation_set(numImgs):
    label_dir =  '/usr/src/datasets/source/c5000v/labels/'
    img_dir = '/usr/src/datasets/source/c5000v/images/'
    src_img_dir = '/usr/src/raw_datasets/coco/val2017'

    remove_files(img_dir,label_dir)

    print('Files removed')

        #coco=COCO(annotations_json_file)
    catIds, cat_map = get_cat_data()
    write_label_files(numImgs, cat_map, catIds, label_dir) 
    copy_images_to_dataset_dir(Path(src_img_dir),Path(img_dir), '.jpg', label_dir) #dataset/images

    os.system(f'ls {img_dir} | wc -l ; ls {label_dir} | wc -l')

def produce_dataset(source_dir, dataset_name):
    # TODO: to abstract the other functions
    return

if __name__ == "__main__":

    coco=COCO('/usr/src/raw_datasets/nightowls/nightowls_training.json')
    #produce_nightowls_training_set(100) #max = 130064

    coco=COCO('/usr/src/raw_datasets/nightowls/nightowls_validation.json')
    #produce_nightowls_validation_set(100)
#
#    annotations_json_file = '/Users/azakaria/Documents/yolo datasets/coco/instances_train2017.json'
#    coco=COCO(annotations_json_file)
#    produce_coco_training_set(480)
#
#    annotations_json_file = '/usr/src/raw_datasets/coco/annotations/instances_val2017.json'
#    coco=COCO(annotations_json_file)
#    produce_coco_validation_set(5000)

loading annotations into memory...
Done (t=0.74s)
creating index...
index created!
loading annotations into memory...
Done (t=0.18s)
creating index...
index created!


In [22]:
catIds, cat_map = get_cat_data()
#print(catIds)
#print(cat_map)

def confirm_files_origin(dir0, dir1):
    #confirm files in dir0 come from dir1
    dir1_images = os.listdir(dir1)
    dir0_images = os.listdir(dir0)
    for f in dir1_images:
        if f not in dir0_images:
            print(f'{f} not in {dir0_images}')
            return 
    print("All image files originate from the correct source dataset")
    print("Note: label files are derived from annotations.json")
    return

#confirm_files_origin('/usr/src/raw_datasets/nightowls/nightowls_validation','/usr/src/datasets/source/n100v/images')

#def test_get_random_img_ids():
    #for v in get_random_img_ids(100):


[7024814,
 7028476,
 7023484,
 7022301,
 7035585,
 7032631,
 7004897,
 7033612,
 7034808,
 7022862,
 7030569,
 7000543,
 7037703,
 7037722,
 7022845,
 7020375,
 7004652,
 7057004,
 7004853,
 7033170,
 7028028,
 7003556,
 7022640,
 7023439,
 7012247,
 7004600,
 7040183,
 7010034,
 7029521,
 7023660,
 7003471,
 7004957,
 7035317,
 7025030,
 7010923,
 7008394,
 7022323,
 7024697,
 7028404,
 7001300,
 7009962,
 7030997,
 7040212,
 7012178,
 7025409,
 7017096,
 7003606,
 7035149,
 7022809,
 7028287,
 7019938,
 7008422,
 7021332,
 7023551,
 7024095,
 7028062,
 7000014,
 7010031,
 7028424,
 7000045,
 7012192,
 7000301,
 7035953,
 7035396,
 7030265,
 7003579,
 7005372,
 7021296,
 7021052,
 7026232,
 7037772,
 7024066,
 7026443,
 7005258,
 7024590,
 7043767,
 7003506,
 7004599,
 7024984,
 7021028,
 7004875,
 7038202,
 7012166,
 7032648,
 7000900,
 7033133,
 7046628,
 7037763,
 7045157,
 7020871,
 7005273,
 7000397,
 7041804,
 7026998,
 7025255,
 7024919,
 7030048,
 7021001,
 7000974,
 7023623]